In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

response = requests.get('https://www.animenewsnetwork.com/encyclopedia/ratings-anime.php?top50=best_bayesian&n=200')

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    tables = soup.find_all('table')

    table = tables[0]

    rows = table.find_all('tr')

    rank = []
    title = []
    rating = []
    votes = []
    format_ = []

    regex = re.compile(r'\([^)]*\)')

    for row in rows[1:]:
        columns = row.find_all('td')

        if len(columns) >= 4:
            rank.append(columns[0].text.strip())
            title_text = columns[1].text.strip()

            if "TV" in title_text:
                format_.append("TV")
            elif "movie" in title_text.lower():
                format_.append("Movie")
            elif "OAV" in title_text or "OVA" in title_text:
                format_.append("OAV")
            else:
                format_.append("Unknown")

            title_text = re.sub(regex, '', title_text)
            title.append(title_text)

            rating.append(columns[2].text.strip())
            votes.append(columns[3].text.strip())

    data = {'Rank': rank, 'Title': title, 'Rating': rating, 'Votes': votes, 'Format': format_}
    df = pd.DataFrame(data)

    df = df.iloc[1:]

    df.reset_index(drop=True, inplace=True)

    print(df)

    df.to_csv('I310D_assignment1_data.csv', index=False)

    from google.colab import files
    files.download('I310D_assignment1_data.csv')

else:
    print(f"Failed to retrieve page, cannot access website. STATUS CODE: {response.status_code}")